In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
    allow_growth=True
    ))

In [2]:
import pandas as pd
import os
target_path = '../data/LOG_G/stable/'
contents = os.listdir(target_path)
data = pd.DataFrame()
num = 0
for c in range(len(contents)):
    print(target_path+contents[c])
    n_data = pd.read_csv(target_path+contents[c],header=None)
    data = pd.concat([data,n_data])

print(len(data))
d = data
#print(ball)
#print(joy)

../data/LOG_G/stable/log201807311344.csv
../data/LOG_G/stable/log201807292107.csv
../data/LOG_G/stable/log201807292213.csv
../data/LOG_G/stable/log201807311352.csv
../data/LOG_G/stable/log201807311349.csv
../data/LOG_G/stable/log201807292116.csv
../data/LOG_G/stable/log201807311336.csv
../data/LOG_G/stable/log201807292215.csv
../data/LOG_G/stable/log201807292207.csv
../data/LOG_G/stable/log201807292110.csv
../data/LOG_G/stable/log201807311347.csv
32853


In [3]:
d.columns = ['l_bx','l_by','l_pole0x','l_pole0y','l_pole1x','l_pole1y','g_bx','g_by','g_pole0x','g_pole0y','g_pole1x','g_pole1y','g_px','g_py','g_pth','ljx','ljy','rjx','rjy']
#d = d[d.bx!=-1]

In [4]:
'''
from sklearn.cross_validation import train_test_split
t1 = []
t2 = []
def f1(x,y):
    return y * np.sin(x) + np.square(x)
def f2(x,y):
    return x * np.cos(y) + np.square(y)
for a in range(1000):
    r = np.array([np.random.rand(), np.random.rand()])
    t1.append(r)
    t2.append(np.array([ f1(r[0],r[1]),f2(r[0],r[1]) ]))
IN_train, IN_test, TAR_train, TAR_test = train_test_split(t1,t2,test_size=0.2,random_state=0)

IN_train = np.array(IN_train)
IN_test = np.array(IN_test)
TAR_train = np.array(TAR_train)
TAR_test = np.array(TAR_test)
print(IN_train[0])
print(IN_test[0])
print(TAR_train[0])
print(TAR_test[0])
print(f1(1,1))
print(f2(1,1))
'''

'\nfrom sklearn.cross_validation import train_test_split\nt1 = []\nt2 = []\ndef f1(x,y):\n    return y * np.sin(x) + np.square(x)\ndef f2(x,y):\n    return x * np.cos(y) + np.square(y)\nfor a in range(1000):\n    r = np.array([np.random.rand(), np.random.rand()])\n    t1.append(r)\n    t2.append(np.array([ f1(r[0],r[1]),f2(r[0],r[1]) ]))\nIN_train, IN_test, TAR_train, TAR_test = train_test_split(t1,t2,test_size=0.2,random_state=0)\n\nIN_train = np.array(IN_train)\nIN_test = np.array(IN_test)\nTAR_train = np.array(TAR_train)\nTAR_test = np.array(TAR_test)\nprint(IN_train[0])\nprint(IN_test[0])\nprint(TAR_train[0])\nprint(TAR_test[0])\nprint(f1(1,1))\nprint(f2(1,1))\n'

In [5]:
from sklearn.cross_validation import train_test_split
l_ball = np.array(d.iloc[:,0:2])
l_pole0 = np.array(d.iloc[:,2:4])
l_pole1 = np.array(d.iloc[:,4:6])
input_d = np.array(d.iloc[:,0:6])
g_ball = np.array(d.iloc[:,6:8])
g_pole0 = np.array(d.iloc[:,8:10])
g_pole1 = np.array(d.iloc[:,10:12])
g_pos = np.array(d.iloc[:,12:14])
output_d = np.array(d.iloc[:,14:18])
ljoy = np.array(d.iloc[:,14:16])
rjoy = np.array(d.iloc[:,16:18])

input_d = np.array(d.iloc[:,0:6])
output_d = np.array(d.iloc[:,14:17])
print(input_d.shape)
print(output_d.shape)

IN_train, IN_test, TAR_train, TAR_test = train_test_split(input_d,output_d,test_size=0.2,random_state=0)
#X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)
#IN_train = IN_train/100.0
#IN_test = IN_test/100.0
for t in range(10):
    print(IN_train[t])
'''
print(IN_train[0])
print(IN_test[0])
print(TAR_train[0])
print(TAR_test[0])
'''

(32853, 6)
(32853, 3)
[ 0.26507283 -0.12912751  1.93868955  2.07983174  2.28975789 -0.49635751]
[ 0.7522718   0.32540024  1.74107202  2.07986238  2.89844302 -0.24833272]
[ 0.40731088 -0.01143569  1.49269182  0.32712971  1.22747513 -2.25930801]
[2.82947013 1.27166372 1.00351973 3.19039456 3.31746766 2.00479555]
[ 3.10863438 -0.73276001  6.68595478  1.98437951  7.59210989 -0.45260283]
[2.67322077 1.01560967 1.01623561 3.08201294 3.22152338 1.7048035 ]
[ 0.44459806  0.38500604  1.49200801  1.93614835  2.3476051  -0.51904077]
[ 0.52533711 -0.0316782   0.88645154  0.96624206  3.01853728 -0.52178025]
[ 0.54904566  0.07638845  1.39258296  1.52994657  1.75798521 -1.04424868]
[2.39541926 0.03300046 3.05722124 2.59769167 3.27572722 0.00688965]


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'\nprint(IN_train[0])\nprint(IN_test[0])\nprint(TAR_train[0])\nprint(TAR_test[0])\n'

In [6]:
#　初期化
sess = tf.Session(config=config)

IN_size = 6
TAR_size = 3
INPUT = tf.placeholder(shape=[None, IN_size], dtype=tf.float32)
TARGET = tf.placeholder(shape=[None, TAR_size], dtype=tf.float32)

In [7]:
#hidden_size = [100]
hidden_size = [20,0,10]
batch_size = 100
iter_size = 70000

#　隠れ層の作成 （last layerに直結）
weights = []
biases = []
layers = []
tmp_size = IN_size
last_layer = INPUT
for hsize in hidden_size:
    weights.append(tf.Variable(tf.random_normal(shape=[tmp_size, hsize], stddev=10.0)))
    biases.append(tf.Variable(tf.random_normal(shape=[hsize], stddev=10.0)))
    layers.append(tf.nn.relu(tf.add(tf.matmul(last_layer, weights[-1]), biases[-1]))) # 活性化関数
    tmp_size = hsize
    last_layer = layers[-1]
    print(layers[-1])
#　出力層
weights.append(tf.Variable(tf.random_normal(shape=[tmp_size, TAR_size], stddev=10.0)))
biases.append(tf.Variable(tf.random_normal(shape=[TAR_size], stddev=10.0)))
layers.append(tf.add(tf.matmul(last_layer, weights[-1]), biases[-1]))
final_output = layers[-1]
print(final_output)

Tensor("Relu:0", shape=(?, 20), dtype=float32)
Tensor("Relu_1:0", shape=(?, 40), dtype=float32)
Tensor("Relu_2:0", shape=(?, 10), dtype=float32)
Tensor("Add_3:0", shape=(?, 3), dtype=float32)


In [ ]:
#　学習
loss = tf.losses.mean_squared_error(TARGET,final_output)
opt = tf.train.AdamOptimizer(0.001)
#opt = tf.train.GradientDescentOptimizer(0.01)
train_step = opt.minimize(loss)
init = tf.global_variables_initializer()
sess.run(init)
loss_vec=[]
for i in range(iter_size):
    rand_index = np.random.choice(len(IN_train), size=batch_size,replace=False)
    rand_in = IN_train[rand_index]
    rand_tar = TAR_train[rand_index]
    sess.run(train_step, feed_dict={INPUT:rand_in, TARGET: rand_tar})
    loss_vec.append(sess.run(loss, feed_dict={INPUT:rand_in, TARGET: rand_tar}))
    if (i+1)%25==0:
#        print(sess.run(INPUT, feed_dict={INPUT:rand_in, TARGET: rand_tar}))
        print('Generation:'+str(i+1)+', Loss = '+str(loss_vec[-1]))

Generation:25, Loss = 1115177400000.0
Generation:50, Loss = 934602340000.0
Generation:75, Loss = 741681200000.0
Generation:100, Loss = 546683200000.0
Generation:125, Loss = 587682550000.0
Generation:150, Loss = 377927300000.0
Generation:175, Loss = 326905270000.0
Generation:200, Loss = 325686660000.0
Generation:225, Loss = 265972560000.0
Generation:250, Loss = 220277310000.0
Generation:275, Loss = 220251180000.0
Generation:300, Loss = 247465700000.0
Generation:325, Loss = 235476660000.0
Generation:350, Loss = 254022450000.0
Generation:375, Loss = 234470490000.0
Generation:400, Loss = 241052420000.0
Generation:425, Loss = 140761200000.0
Generation:450, Loss = 153475060000.0
Generation:475, Loss = 138158130000.0
Generation:500, Loss = 199649150000.0
Generation:525, Loss = 196680880000.0
Generation:550, Loss = 154896060000.0
Generation:575, Loss = 103024740000.0
Generation:600, Loss = 173370590000.0
Generation:625, Loss = 113918610000.0
Generation:650, Loss = 143554600000.0
Generation:675

Generation:5550, Loss = 3194178300.0
Generation:5575, Loss = 3140355800.0
Generation:5600, Loss = 5108646000.0
Generation:5625, Loss = 4624254500.0
Generation:5650, Loss = 3347721200.0
Generation:5675, Loss = 4250792400.0
Generation:5700, Loss = 4074637600.0
Generation:5725, Loss = 3511181300.0
Generation:5750, Loss = 3504524800.0
Generation:5775, Loss = 4353233400.0
Generation:5800, Loss = 3092093400.0
Generation:5825, Loss = 3048283000.0
Generation:5850, Loss = 3996562700.0
Generation:5875, Loss = 3293287400.0
Generation:5900, Loss = 3310852400.0
Generation:5925, Loss = 2647519000.0
Generation:5950, Loss = 2704757500.0
Generation:5975, Loss = 2377816600.0
Generation:6000, Loss = 2807934500.0
Generation:6025, Loss = 2881334500.0
Generation:6050, Loss = 2121442700.0
Generation:6075, Loss = 4550708000.0
Generation:6100, Loss = 2758125000.0
Generation:6125, Loss = 2480029700.0
Generation:6150, Loss = 2409795800.0
Generation:6175, Loss = 3950910000.0
Generation:6200, Loss = 2349889300.0
G

Generation:11175, Loss = 357824800.0
Generation:11200, Loss = 644269700.0
Generation:11225, Loss = 499075200.0
Generation:11250, Loss = 446198850.0
Generation:11275, Loss = 459873150.0
Generation:11300, Loss = 500454200.0
Generation:11325, Loss = 356097200.0
Generation:11350, Loss = 453690180.0
Generation:11375, Loss = 443440930.0
Generation:11400, Loss = 581257340.0
Generation:11425, Loss = 390664450.0
Generation:11450, Loss = 408456350.0
Generation:11475, Loss = 440861150.0
Generation:11500, Loss = 609770050.0
Generation:11525, Loss = 469838800.0
Generation:11550, Loss = 384896960.0
Generation:11575, Loss = 427307260.0
Generation:11600, Loss = 384497570.0
Generation:11625, Loss = 379671680.0
Generation:11650, Loss = 425796800.0
Generation:11675, Loss = 485789630.0
Generation:11700, Loss = 408405920.0
Generation:11725, Loss = 455573950.0
Generation:11750, Loss = 473328580.0
Generation:11775, Loss = 371967170.0
Generation:11800, Loss = 395040130.0
Generation:11825, Loss = 411048060.0
G

Generation:16725, Loss = 115623220.0
Generation:16750, Loss = 105982940.0
Generation:16775, Loss = 139455060.0
Generation:16800, Loss = 123251540.0
Generation:16825, Loss = 120163544.0
Generation:16850, Loss = 134400050.0
Generation:16875, Loss = 157898460.0
Generation:16900, Loss = 136987250.0
Generation:16925, Loss = 175567630.0
Generation:16950, Loss = 121877150.0
Generation:16975, Loss = 140031540.0
Generation:17000, Loss = 126643640.0
Generation:17025, Loss = 136517390.0
Generation:17050, Loss = 129584700.0
Generation:17075, Loss = 125840290.0
Generation:17100, Loss = 102580990.0
Generation:17125, Loss = 258951090.0
Generation:17150, Loss = 125095050.0
Generation:17175, Loss = 134299790.0
Generation:17200, Loss = 128041500.0
Generation:17225, Loss = 141485100.0
Generation:17250, Loss = 193450660.0
Generation:17275, Loss = 134230060.0
Generation:17300, Loss = 106267736.0
Generation:17325, Loss = 113061470.0
Generation:17350, Loss = 112987220.0
Generation:17375, Loss = 150901460.0
G

Generation:22425, Loss = 69338520.0
Generation:22450, Loss = 63212484.0
Generation:22475, Loss = 49316464.0
Generation:22500, Loss = 71424630.0
Generation:22525, Loss = 71788584.0
Generation:22550, Loss = 53784076.0
Generation:22575, Loss = 52353410.0
Generation:22600, Loss = 64180584.0
Generation:22625, Loss = 68956460.0
Generation:22650, Loss = 47136652.0
Generation:22675, Loss = 68177030.0
Generation:22700, Loss = 51558004.0
Generation:22725, Loss = 70632120.0
Generation:22750, Loss = 55602980.0
Generation:22775, Loss = 65627790.0
Generation:22800, Loss = 54919972.0
Generation:22825, Loss = 48057656.0
Generation:22850, Loss = 59160548.0
Generation:22875, Loss = 65508624.0
Generation:22900, Loss = 57339964.0
Generation:22925, Loss = 64294092.0
Generation:22950, Loss = 54260844.0
Generation:22975, Loss = 39122396.0
Generation:23000, Loss = 58493092.0
Generation:23025, Loss = 60592652.0
Generation:23050, Loss = 47771830.0
Generation:23075, Loss = 52130150.0
Generation:23100, Loss = 805

Generation:28125, Loss = 32619226.0
Generation:28150, Loss = 25119468.0
Generation:28175, Loss = 22301644.0
Generation:28200, Loss = 29571206.0
Generation:28225, Loss = 28616502.0
Generation:28250, Loss = 31732668.0
Generation:28275, Loss = 36919070.0
Generation:28300, Loss = 26775066.0
Generation:28325, Loss = 31967754.0
Generation:28350, Loss = 47751900.0
Generation:28375, Loss = 50414416.0
Generation:28400, Loss = 25846148.0
Generation:28425, Loss = 26244628.0
Generation:28450, Loss = 25924042.0
Generation:28475, Loss = 24612428.0
Generation:28500, Loss = 26003438.0
Generation:28525, Loss = 29272614.0
Generation:28550, Loss = 35120170.0
Generation:28575, Loss = 32382792.0
Generation:28600, Loss = 27940928.0
Generation:28625, Loss = 28757654.0
Generation:28650, Loss = 39216596.0
Generation:28675, Loss = 31943850.0
Generation:28700, Loss = 32179510.0
Generation:28725, Loss = 33466682.0
Generation:28750, Loss = 25614182.0
Generation:28775, Loss = 36792680.0
Generation:28800, Loss = 350

Generation:33875, Loss = 27614444.0
Generation:33900, Loss = 22722028.0
Generation:33925, Loss = 20608986.0
Generation:33950, Loss = 17291568.0
Generation:33975, Loss = 22707562.0
Generation:34000, Loss = 24332588.0
Generation:34025, Loss = 26918720.0
Generation:34050, Loss = 20558116.0
Generation:34075, Loss = 37715630.0
Generation:34100, Loss = 27313650.0
Generation:34125, Loss = 26545928.0
Generation:34150, Loss = 16265557.0
Generation:34175, Loss = 24383290.0
Generation:34200, Loss = 26858280.0
Generation:34225, Loss = 18911856.0
Generation:34250, Loss = 34586116.0
Generation:34275, Loss = 27585614.0
Generation:34300, Loss = 22211390.0


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.grid()
plt.plot(loss_vec)
#plt.xlim(0,25)
plt.ylim(0,2)
ax.set_xlabel('iteration')
ax.set_ylabel('loss')
plt.savefig(str(hidden_size)+'.png')

In [ ]:
# 重みの保存
saver = tf.train.Saver()
saver.save(sess, './model.ckpt')


In [ ]:
'''
# ボール座標とスティックの傾きプロット
by_max=IN_train.max(axis=0)[1]
by_min=IN_train.min(axis=0)[1]

ball_y = np.array(range(int(by_min)-3,int(by_max)+3))
for i in range(len(ball_y)):
    
    ball = np.array([[6.0, ball_y[i]/10.0]])
    
    print(ball)
    joy = sess.run(final_output, feed_dict={INPUT: ball})
    plt.scatter(ball[0][1],ball[0][0], s=200, c='orange')
    
    plt.quiver(0,0,-joy[0][0],-joy[0][1],angles="xy",scale_units="xy",scale=1)
    #plt.quiver(0,0,-0.6,5,angles="xy",scale_units="xy",scale=1)
    print(joy)
    plt.savefig(str(hidden_size)+'_img'+'.png')
    '''

In [ ]:
# 重みのrestore
predicts_sess = tf.Session(config=config)
ckpt = tf.train.get_checkpoint_state('./')
if ckpt == None:
    print('!!!!!!!WEIGHTS NOT FOUND!!!!!!')

saver.restore(predicts_sess, './model.ckpt')